In [1]:
from glob import glob
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd

import torch

from matplotlib import pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertModel

## abhishek の fold を確認する

In [2]:
folds_df = pd.read_csv('../inputs/datasets/abhishek_folds/train_folds.csv')
folds_df

,textID,text,selected_text,sentiment,kfold
0,f7fdea625a,i`m so bored i can barely even tweet. i have n...,bored,negative,0
1,c19de2c75b,awwwwwwwwww thats jus...awwwww.did she get t...,awwwwwwwwww thats jus...awwwww.did she get to...,neutral,0
2,0f963af18f,I did not twitt yesterday cause it was a very ...,I can not sleep,negative,0
3,0583c78cc1,Congratulation`s to phil packer on completing ...,Congratulation`s,positive,0
4,1cdb444ea5,O`Charleys? Pretty good. Especially when its f...,Pretty good.,positive,0
...,...,...,...,...,...
27475,0c88d19f11,I think it`s time to take a shower & get ready...,I think it`s time to take a shower & get ready...,neutral,4
27476,ab8d203651,thanks love ) btw happy mother`s day to your mom,thanks love ) btw happy mother`s day to your mom,positive,4
27477,430095620f,Theres a fricken prisnor trans outside r hotel...,Theres a fricken prisnor trans outside r hotel...,neutral,4
27478,3c66250da9,to the people i was txting... sorry phone is d...,to the people i was txting... sorry phone is d...,neutral,4


In [3]:
# まぁ sentiment 的には ok かなぁ
pd.crosstab(folds_df.sentiment, folds_df.kfold)

kfold,0,1,2,3,4
sentiment,,,,,
negative,1557,1556,1556,1556,1556
neutral,2223,2224,2224,2223,2223
positive,1716,1716,1716,1717,1717


## e045 の valid を確認する

In [4]:
trn_df = pd.read_csv('../inputs/origin/train.csv')
display(trn_df.shape, trn_df.head())

(27481, 4)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
trn_df = trn_df[trn_df.selected_text.notnull()]
trn_df.shape

(27480, 4)

In [6]:
!du -sh ../checkpoints/*

8.0K	../checkpoints/e001
4.0K	../checkpoints/e002
487M	../checkpoints/e005
28K	../checkpoints/e006
7.5G	../checkpoints/e007
28K	../checkpoints/e009
8.0K	../checkpoints/e010
4.0K	../checkpoints/e011
4.0K	../checkpoints/e013
4.0K	../checkpoints/e014
4.0K	../checkpoints/e019
4.0K	../checkpoints/e020
4.0K	../checkpoints/e021
4.0K	../checkpoints/e022
4.0K	../checkpoints/e023
4.0K	../checkpoints/e025
7.0G	../checkpoints/e026
1.4G	../checkpoints/e030
7.0G	../checkpoints/e031
8.0K	../checkpoints/e032
4.0K	../checkpoints/e033
7.0G	../checkpoints/e034
8.0K	../checkpoints/e035
8.0K	../checkpoints/e037
1.4G	../checkpoints/e038
4.0K	../checkpoints/e040
8.0K	../checkpoints/e041
7.0G	../checkpoints/e042
4.0K	../checkpoints/e043
4.0K	../checkpoints/e044
7.0G	../checkpoints/e045
7.0G	../checkpoints/e048
8.0K	../checkpoints/e049
8.0K	../checkpoints/e051
12G	../checkpoints/e052
4.0K	../checkpoints/e055
7.0G	../checkpoints/e056
4.0K	../checkpoints/e057
2.8G	../checkpoints/e058
7.0G	../checkpoints/e059
7.0

In [7]:
ckpts = []
for fckpt in glob('../checkpoints/e059/best/*'):
    ckpts.append(torch.load(fckpt))

In [8]:
ckpts[0].keys()

dict_keys(['fold_num', 'current_epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'val_textIDs', 'val_input_ids', 'val_preds', 'val_labels', 'histories'])

In [9]:
def _get_predicted_texts(input_ids, y_preds_head, y_preds_tail, tokenizer):
    predicted_texts = []
    for input_id, y_pred_head, y_pred_tail \
            in zip(input_ids, y_preds_head, y_preds_tail):
        pred_label_head = y_pred_head.argmax()
        pred_label_tail = y_pred_tail.argmax()
        predicted_text = tokenizer.decode(
            input_id[pred_label_head:pred_label_tail])
        predicted_texts.append(predicted_text)

    return predicted_texts

In [10]:
def _get_selected_texts(input_ids, labels_head, labels_tail, tokenizer):
    predicted_texts = []
    for input_id, pred_label_head, pred_label_tail \
            in zip(input_ids, labels_head, labels_tail):
        predicted_text = tokenizer.decode(
            input_id[pred_label_head:pred_label_tail])
        predicted_texts.append(predicted_text)

    return predicted_texts

In [11]:
import sys
sys.path.append('../')
from tools.tokenizers import myRobertaByteLevelBPETokenizer

tokenizer = myRobertaByteLevelBPETokenizer(   
    vocab_file='../inputs/datasets/roberta/tokenizer/vocab.json',
    merges_file='../inputs/datasets/roberta/tokenizer/merges.txt',
    lowercase=True,
    add_prefix_space=True)

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
for ckpt in ckpts:
    trn_df = trn_df.set_index('textID')
    predicted_texts = _get_predicted_texts(ckpt['val_input_ids'], ckpt['val_preds'][0],  ckpt['val_preds'][1], tokenizer)
    trn_df.loc[ckpt['val_textIDs'], 'predicted_texts'] = predicted_texts
    trn_df = trn_df.reset_index()
trn_df

,textID,text,selected_text,sentiment,predicted_texts
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i`d have responded, if i were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,bullying me...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of ****,"
...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,lost his job and can`t afford it
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,i`ve wondered
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,but it was worth it ****.


In [13]:
for ckpt in ckpts:
    trn_df = trn_df.set_index('textID')
    manual_selected_texts = _get_selected_texts(ckpt['val_input_ids'], ckpt['val_labels'][0],  ckpt['val_labels'][1], tokenizer)
    trn_df.loc[ckpt['val_textIDs'], 'manual_selected_text'] = manual_selected_texts
    trn_df = trn_df.reset_index()
trn_df

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i`d have responded, if i were going","i`d have responded, if i were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,bullying me...,bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,leave me alone,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of ****,","sons of ****,"
...,...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,lost his job and can`t afford it,husband lost
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,i`ve wondered,", don`t force"
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good,yay good for both of you.
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,but it was worth it ****.,but it was worth it ****.


In [14]:
trn_df.query('textID == "6cd1ba8680"')

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text
19561,6cd1ba8680,Thats charmingly funny,ats charmingly fun,positive,thats charmingly funny,thats charmingly funny</s>


In [15]:
trn_df[trn_df.manual_selected_text.str.contains('</s>')]

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text
636,b04bbb81c9,Hi to one kiwi artist from another kiwi artist,Hi to one kiwi artist from another kiwi artis,neutral,hi to one kiwi artist from another kiwi artist,hi to one kiwi artist from another kiwi artis...
638,289317384e,oooo ok why havent you accepted my friends...,oooo ok why havent you accepted my friends ...,neutral,oooo ok why havent you accepted my friends re...,oooo ok why havent you accepted my friends re...
1419,6e675117b8,Me too haha I did but have no credit,Me too haha I did but have no credi,neutral,me too haha i did but have no credit,me too haha i did but have no credit</s>
1753,4ff8b3efb4,I can`t has blocked me. I can`t even request,I can`t has blocked me. I can`t even reques,negative,i can`t has blocked,i can`t has blocked me. i can`t even request</s>
2391,f530f15848,I love all my friends and drew were fun t...,I love all my friends and drew were fun toni,positive,i love,i love all my friends and drew were fun tonig...
4368,ff207f2540,More to thank for #ff shouts and heaps mor...,More to thank for #ff shouts and heaps mor...,neutral,more to thank for #ff shouts and heaps more. ...,more to thank for #ff shouts and heaps more. ...
6260,b056ec3a6f,"hey y`all, we are waiting for final edits on...","hey y`all, we are waiting for final edits on o...",neutral,"hey y`all, we are waiting for final edits on ...","hey y`all, we are waiting for final edits on ..."
7441,d12148ecc8,I need a wardrobe intervention,I need a wardrobe interventio,neutral,i need a wardrobe intervention,i need a wardrobe intervention</s></s>
7836,b30b442df8,invite sent You can now invite more collabs...,invite sent You can now invite more collabs ...,neutral,invite sent you can now invite more collabs t...,invite sent you can now invite more collabs t...
8248,27eed86c41,hi I`m ok still not feeling great,hi I`m ok still not feeling grea,negative,not feeling great,hi i`m ok still not feeling great</s>


In [16]:
trn_df['selected_text_lower'] = trn_df['selected_text'].apply(lambda x: " " + " ".join(x.lower().split()))
trn_df

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text,selected_text_lower
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i`d have responded, if i were going","i`d have responded, if i were going","i`d have responded, if i were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad,sooo sad,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,bullying me...,bullying me,bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,leave me alone,leave me alone,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of ****,","sons of ****,","sons of ****,"
...,...,...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,lost his job and can`t afford it,husband lost,d lost
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,i`ve wondered,", don`t force",", don`t force"
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good,yay good for both of you.,yay good for both of you.
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,but it was worth it ****.,but it was worth it ****.,but it was worth it ****.


In [17]:
(trn_df.selected_text_lower == trn_df.manual_selected_text).mean()

0.887117903930131

In [18]:
trn_df[(trn_df.selected_text_lower != trn_df.manual_selected_text)].query('sentiment != "neutral"').head(100)

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text,selected_text_lower
18,af3fed7fc3,is back home now gonna miss every one,onna,negative,miss,is back,onna
27,bdc32ea43c,On the way to Malaysia...no internet access to...,.no internet,negative,no internet access to twit,...no internet,.no internet
32,1c31703aef,If it is any consolation I got my BMI tested ...,well so much for being unhappy for about 10 mi...,negative,unhappy,ed well so much for being unhappy for about 10,well so much for being unhappy for about 10 m...
39,2863f435bd,A little happy for the wine jeje ok it`sm my f...,A little happy fo,positive,love this day,a little happy for,a little happy fo
48,3d9d4b0b55,"i donbt like to peel prawns, i also dont like ...",dont like go,negative,"i donbt like to peel prawns, i also dont like",also dont like,dont like go
...,...,...,...,...,...,...,...
949,55f2673e6b,Well thatï¿½s disappointing to hear.,s disappointing,negative,disappointing,s disappointing,s disappointing
955,8ac49207e1,(cont) a bastardized version of French? It`s ...,"s fun,",positive,"it`s fun,","s fun,","s fun,"
956,c400a6bf99,"alas, I am moving (like where i`m moving to...",h) wish I could go too,positive,wish,gh) wish i could go too,h) wish i could go too
962,e0ef3a78fc,mad the rain got me...now i cant go see jaiden...,mad the rain got me..,negative,mad,mad the rain got me...,mad the rain got me..


In [19]:
trn_df.shape, trn_df[(trn_df.selected_text_lower != trn_df.manual_selected_text)].shape

((27480, 7), (3102, 7))

In [20]:
trn_df['manual_and_selected_intersection_len'] = trn_df.apply(lambda row: len(set(row['manual_selected_text'].split()) & set(row['selected_text_lower'].split())), axis=1)
trn_df['manual_and_selected_intersection_len'].value_counts().sort_index()

0      505
1     7121
2     2331
3     2134
4     1968
5     1592
6     1367
7     1171
8     1002
9      876
10     814
11     732
12     661
13     584
14     553
15     507
16     464
17     457
18     430
19     396
20     415
21     331
22     316
23     290
24     210
25     114
26      77
27      40
28      14
29       8
Name: manual_and_selected_intersection_len, dtype: int64

In [21]:
trn_df['manual_and_selected_tokenized_intersection_len'] = trn_df.apply(lambda row: len(set(tokenizer.encode(' '.join(row['manual_selected_text'].split()), add_special_tokens=False).ids) & set(tokenizer.encode(' '.join(row['selected_text_lower'].split()), add_special_tokens=False).ids)), axis=1)
trn_df['manual_and_selected_tokenized_intersection_len'].value_counts().sort_index()

0      134
1     5182
2     2836
3     1840
4     1750
5     1606
6     1318
7     1148
8     1029
9      945
10     796
11     711
12     669
13     659
14     575
15     498
16     516
17     480
18     444
19     446
20     381
21     363
22     374
23     355
24     363
25     318
26     306
27     279
28     241
29     238
30     165
31     128
32     134
33      88
34      56
35      29
36      25
37      16
38      15
39       8
40       8
41       5
42       3
Name: manual_and_selected_tokenized_intersection_len, dtype: int64

In [22]:
trn_df.query('manual_and_selected_tokenized_intersection_len == 0')

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text,selected_text_lower,manual_and_selected_intersection_len,manual_and_selected_tokenized_intersection_len
18,af3fed7fc3,is back home now gonna miss every one,onna,negative,miss,is back,onna,0,0
168,e56eeb4968,Few Bevvies 2day in twn..great on a day off!!,great,positive,great,few,great,0,0
189,a0a306868a,lost my tooth 2day whilst i was eating gum...oww,oww,negative,lost,lost my,oww,0,0
309,a54d3c2825,"I know It was worth a shot, though!",as wort,positive,worth,i know it,as wort,0,0
348,322b61740c,"90 degrees, gross skies, and thunderstorms...p...",perfect ma,positive,perfect,90 degrees,perfect ma,0,0
...,...,...,...,...,...,...,...,...,...
26975,7b11a02202,We were all set to get a room last Saturday...,or missin,negative,i can`t apologise enough,we were all,or missin,0,0
27120,a11ddb9e58,The geographY was an exam today!But turned out...,ry nervou,negative,nervous,the geography was an,ry nervou,0,0
27208,30a1e8c2b4,Guys ( ) I know! My ability to read time te...,es faile,negative,failed!,guys ( ),es faile,0,0
27375,13fbc75291,Going to church in the morning...happy mommas ...,happy,positive,happy,going,happy,0,0


In [23]:
trn_df.to_csv('../inputs/nes_info/my_dataset_trn_df.csv', index=False)

In [90]:
trn_df.loc[955]['text']

' (cont) a bastardized version of French? It`s fun, believe me!'

In [86]:
trn_df.loc[955]['selected_text_lower']

' s fun,'

In [88]:
trn_df.loc[955]['manual_selected_text']

's fun,'

In [81]:
tokenizer.encode('i also dont like going shopping').tokens

['Ġi', 'Ġalso', 'Ġdont', 'Ġlike', 'Ġgoing', 'Ġshopping']

In [65]:
trn_df.loc[27478]

textID                                   ed167662a5
text                     But it was worth it  ****.
selected_text            But it was worth it  ****.
sentiment                                  positive
predicted_texts           but it was worth it ****.
manual_selected_text      but it was worth it ****.
selected_text_lower      but it was worth it  ****.
Name: 27478, dtype: object

In [66]:
trn_df.loc[27478]['manual_selected_text']

' but it was worth it ****.'

In [68]:
trn_df.loc[27478]['selected_text_lower']

' but it was worth it  ****.'

In [69]:
trn_df.loc[27478]['manual_selected_text'] == trn_df.loc[27478]['selected_text_lower']

False

In [70]:
for i, j in zip(trn_df.loc[27478]['manual_selected_text'], trn_df.loc[27478]['selected_text_lower']):
    print(i, j)

   
b b
u u
t t
   
i i
t t
   
w w
a a
s s
   
w w
o o
r r
t t
h h
   
i i
t t
   
*  
* *
* *
* *
. *


## debug

In [27]:
row = trn_df.query('textID == "6cd1ba8680"').iloc[0]

In [36]:
text = " " + " ".join(row['text'].split())
text_output = tokenizer.encode_plus(
    text=text,
    # text_pair=None,
    # text_pair=f"[{row['sentiment']}]",
    text_pair=row['sentiment'],
    add_special_tokens=True,
    max_length=125,
    pad_to_max_length=True,
    return_tensor='pt',
    return_token_type_ids=False,
    return_attention_mask=True,
    return_special_tokens_mask=True,
)
row['input_ids'] = text_output['input_ids']
row['attention_mask'] = text_output['attention_mask']
row['special_tokens_mask'] = text_output['special_tokens_mask']

if 'selected_text' not in row:
    row['selected_text'] = ''
    row['labels_head'] = -1
    row['labels_tail'] = -1
#    return row

text = " " + " ".join(row['selected_text'].split())
selected_text_output = tokenizer.encode_plus(
    text=text,
    text_pair=None,
    add_special_tokens=False,
    max_length=125,
    pad_to_max_length=False,
    return_tensor='pt',
    return_token_type_ids=False,
    return_attention_mask=False,
)

# allign labels for segmentation
input_ids = text_output['input_ids']
sel_input_ids = selected_text_output['input_ids']
# 1 start なのは、先頭の token をスルーするため
matched_cnt = len([i for i in input_ids[1:1 + len(sel_input_ids)]
                   if i in sel_input_ids])
best_matched_cnt = matched_cnt
best_matched_i = 1
# for i in range(0, len(input_ids)):
# 1 start なのは、先頭の token をスルーするため
for i in range(1, len(input_ids) - len(sel_input_ids)):
    head_input_id_i = input_ids[i]
    tail_input_id_i = input_ids[i + len(sel_input_ids)]
    if head_input_id_i in sel_input_ids:
        matched_cnt -= 1
    if tail_input_id_i in sel_input_ids:
        matched_cnt += 1
    if matched_cnt < 0:
        raise Exception('invalid logic')

    if best_matched_cnt < matched_cnt:
        best_matched_cnt = matched_cnt
        best_matched_i = i + 1   # 抜いた時の話なので
    if best_matched_cnt == len(sel_input_ids):
        break

row['labels_head'] = best_matched_i
row['labels_tail'] = best_matched_i + len(sel_input_ids)

In [44]:
row['text']

'  Thats charmingly funny'

In [45]:
row['selected_text']

'ats charmingly fun'

In [39]:
selected_text_output['input_ids']

[23, 29, 18452, 352, 1531]

In [40]:
" " + " ".join(row['selected_text'].split())

' ats charmingly fun'

In [42]:
tokenizer.encode(" " + " ".join(row['selected_text'].split())).ids

[23, 29, 18452, 352, 1531]

In [43]:
tokenizer.encode(" " + " ".join(row['text'].split())).ids

[45365, 18452, 352, 6269]

In [ ]:
tokenizer.decode()

## tokenizer 確認

In [17]:
import sys
sys.path.append('../')
from tools.tokenizers import myRobertaByteLevelBPETokenizer

my_tokenizer = myRobertaByteLevelBPETokenizer(   
    vocab_file='../inputs/datasets/roberta/tokenizer/vocab.json',
    merges_file='../inputs/datasets/roberta/tokenizer/merges.txt',
    lowercase=True,
    add_prefix_space=True)

In [21]:
my_tokenizer.decode(ckpt['val_input_ids'][0][1:11])

' i`d have responded, if i were going'

In [52]:
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [53]:
temp_jac = 0
for i, row in trn_df.iterrows():
    temp_jac += jaccard(row['selected_text'], row['predicted_texts'])

In [54]:
temp_jac / len(trn_df)

0.44581498228808364

## 改善の必要あり
 - decode すると違う形式になってしまう
 - selected_text を encode -> decode して形式が変わるパターンを除去する後処理が必要そう

In [75]:
trn_df.loc[0]

textID                                                         cb774db0d1
text                                  I`d have responded, if I were going
selected_text                         I`d have responded, if I were going
sentiment                                                         neutral
predicted_texts         i ` d have responded, if i were going [SEP] [PAD]
manual_selected_text    i ` d have responded, if i were going [SEP] [PAD]
Name: 0, dtype: object

In [74]:
jaccard(trn_df.loc[0]['selected_text'], trn_df.loc[0]['predicted_texts'][:-12])

0.6666666666666666

In [72]:
trn_df.loc[0]['selected_text'][4:], trn_df.loc[0]['predicted_texts'][6:-12]

('have responded, if I were going', 'have responded, if i were going')